In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statistics
import sklearn.metrics as metrics
import tensorflow as tf
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.feature_selection import RFECV
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.ensemble import RandomForestClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("../new_train.csv")
test = pd.read_csv("../new_test.csv")

In [3]:
train

,Unnamed: 0,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,...,max_url_per_auction,min_url_per_auction,std_url_per_auction,total_no_of_participated_auctions,no_of_auction_exceeds_threshold,percentage_of_auctions_above_threshold,total_no_of_bidded_category,no_of_merchandise_exceeds_threshold,percentage_of_merchandise_above_threshold,on_url_that_has_a_bot_mean
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,...,1.0,1.0,0.000000,18.0,0.0,0.000000,1.0,0.0,0.0,1.000000
1,1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,...,2.0,2.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.500000
2,2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,...,1.0,1.0,0.000000,4.0,0.0,0.000000,1.0,0.0,0.0,0.500000
3,3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,1.000000
4,4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,...,21.0,1.0,5.644263,23.0,1.0,0.043478,1.0,0.0,0.0,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2008,369515b3af4f8ca582f90271d30b14b6r52aw,a1f85275793c4a782f0a668711f41b927ivc9,e6882cf204a9482edd042b6e31791dfctxzx8,0.0,25.0,1.0,4.0,33.0,4.0,...,1.0,1.0,0.000000,25.0,1.0,0.040000,1.0,0.0,0.0,0.500000
2009,2009,f939c17ffc7c39ac9b35b69e5e75179fv9pe2,a3d2de7675556553a5f08e4c88d2c2286s1m2,b9b03d5a127eb07aeb9163cdcf524e1344ac9,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2010,2010,c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl,a3d2de7675556553a5f08e4c88d2c22856leq,d02c2b288b8aabd79ff47118aff41a2dqwzwc,0.0,1.0,1.0,2.0,2.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000
2011,2011,0381a69b7a061e9ace2798fd48f1f537mgq57,fd87037ce0304077079c749f420f0b4c54uo0,f030a221726fbcdfc4dc7dfd1b381a112hieq,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.0,0.000000


In [4]:
train.drop(train.filter(regex="Unname"),axis=1, inplace=True)
test.drop(test.filter(regex="Unname"),axis=1, inplace=True)

In [5]:
X = train.drop(columns=['bidder_id', 'payment_account', 'address', 'outcome','merchandise']) 
y = train['outcome']
X_test_original = test.drop(columns=['bidder_id', 'payment_account', 'address', 'merchandise'])

### Scaling the training and test dataset

In [6]:
X.shape

(2013, 52)

In [7]:
X.columns

Index(['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'auto parts', 'books and music', 'clothing',
       'computers', 'furniture', 'home goods', 'jewelry', 'mobile',
       'office equipment', 'sporting goods', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_excee

In [8]:
scaler = StandardScaler()
# scaled_features = data.copy()
col_names = ['auction', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'num_first_bids', 'num_last_bids', 'time_to_bid', 'inst_resp',
       'perc_inst_resp', 'num_bids_per_auction',
       'num_bids_per_device', 'num_bids_per_country', 'num_bids_per_ip',
       'on_ip_that_has_a_bot_mean', 'ip_entropy', 'url_entropy',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction',
       'total_no_of_participated_auctions', 'no_of_auction_exceeds_threshold',
       'percentage_of_auctions_above_threshold', 'total_no_of_bidded_category',
       'no_of_merchandise_exceeds_threshold',
       'percentage_of_merchandise_above_threshold',
       'on_url_that_has_a_bot_mean']

train_features = X[col_names]
scaler = StandardScaler().fit(train_features.values)
train_features = scaler.transform(train_features.values)
X[col_names] = train_features

test_features = X_test_original[col_names]
scaler_test = StandardScaler().fit(test_features.values)
test_features = scaler_test.transform(test_features.values)
X_test_original[col_names] = test_features

In [9]:
selected_features = ['num_bids_per_ip', 'num_bids_per_auction', 'num_bids_per_country', 'mean_ip_per_auction', 'perc_inst_resp', 'time', 'mean_url_per_auction', 'inst_resp', 'num_bids', 'time_to_bid']
X = X[selected_features]
X_test_original = X_test_original[selected_features]

### Helper Functions

In [10]:
def evaluate(model, X_train, y_train, X_test, y_test):
    print("TRAIN")
    train_predictions = model.predict_proba(X_train)
    train_pred = model.predict(X_train)
        
    train_accuracy = accuracy_score(y_train, train_pred)
    train_auc_roc_score = roc_auc_score(y_train,train_predictions[:,1])
    train_fbeta = fbeta_score(y_train, train_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_train, train_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_train, train_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(train_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(train_auc_roc_score))
    print("*" * 100)
    
    print("TEST")
    
    test_predictions = model.predict_proba(X_test)
    test_pred = model.predict(X_test)
        
    test_accuracy = accuracy_score(y_test, test_pred)
    test_auc_roc_score = roc_auc_score(y_test,test_predictions[:,1])
    test_fbeta = fbeta_score(y_test, test_pred, average='binary', beta=2.0)
    
    print("Classification report")
    print(classification_report(y_test, test_pred, digits = 4))
    
    print("FBeta Score")
    print(fbeta_score(y_test, test_pred, average='binary', beta=2.0))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(test_accuracy))
    print('AUC ROC = {:0.4f}%.'.format(test_auc_roc_score))
    print("*" * 100)
    
    return [train_accuracy, train_auc_roc_score, train_fbeta, test_accuracy, test_auc_roc_score, test_fbeta]

In [11]:
def process_results(arr):
    train_accuracy = []
    train_auc_roc_score = [] 
    train_fbeta = []
    test_accuracy = []
    test_auc_roc_score = []
    test_fbeta = []
    
    for item in arr:
        train_accuracy.append(item[0])
        train_auc_roc_score.append(item[1])
        train_fbeta.append(item[2])
        test_accuracy.append(item[3])
        test_auc_roc_score.append(item[4])
        test_fbeta.append(item[5])
    
    mean_accuracy = np.array(train_accuracy).mean()
    mean_train_auc_roc_score = np.array(train_auc_roc_score).mean()
    mean_train_fbeta = np.array(train_fbeta).mean()
    mean_test_accuracy = np.array(test_accuracy).mean()
    mean_test_auc_roc_score = np.array(test_auc_roc_score).mean()
    mean_test_fbeta = np.array(test_fbeta).mean()
    
    print("final train accuracy: " + str(mean_accuracy))
    print("final train AUC: " + str(mean_train_auc_roc_score))
    print("final train fbeta: " + str(mean_train_fbeta))
    print("final test accuracy: " + str(mean_test_accuracy))
    print("final test AUC: " + str(mean_test_auc_roc_score))
    print("final test fbeta: " + str(mean_test_fbeta))

### SKFold + SMOTE (Base Model)

In [12]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
counter = 1
ann_result = []

over = SMOTE(sampling_strategy=0.2, random_state = 42)
under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

for train_index, test_index in skf.split(X, y):
    print("Fold number " + str(counter))
    counter += 1
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    # ann
    print("ANN")
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann.fit(X_train, y_train)
    ann_result.append(evaluate(ann, X_train, y_train, X_test, y_test))

Fold number 1
ANN


2021-11-07 18:14:37.325920: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 18:14:37.380559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.7759    0.9633    0.8595       762
         1.0     0.7686    0.3049    0.4366       305

    accuracy                         0.7751      1067
   macro avg     0.7722    0.6341    0.6481      1067
weighted avg     0.7738    0.7751    0.7386      1067

FBeta Score
0.3467561521252796
Model Performance
Accuracy = 0.7751%.
AUC ROC = 0.8210%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9515    0.9764    0.9638       382
         1.0     0.1818    0.0952    0.1250        21

    accuracy                         0.9305       403
   macro avg     0.5667    0.5358    0.5444       403
weighted avg     0.9114    0.9305    0.9201       403

FBeta Score
0.10526315789473684
Model Performance
Accuracy = 0.9305%.
AUC ROC = 0.7760%.
*************

In [13]:
process_results(ann_result)

final train accuracy: 0.7671977507029054
final train AUC: 0.8096717008734565
final train fbeta: 0.3056062984449682
final test accuracy: 0.9269693715047591
final test AUC: 0.7960072301171777
final test fbeta: 0.19049998977630556


### SKFold + SMOTE (Random Search)

In [14]:
def random_search_ann(X, y):
    print("RANDOM SEARCH ANN EXPERIMENT")
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]

    ann_random_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)

    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_random = RandomizedSearchCV(estimator = ann, param_distributions = ann_random_grid, n_iter = 100, cv = skf, 
                                verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    ann_random.fit(X_train, y_train)
    ann_best_random = ann_random.best_estimator_
    ann_random_accuracy = evaluate(ann_best_random, X_train, y_train, X_test, y_test)
    print("Best Random Search Param for ANN")
    print(ann_random.best_params_)

In [15]:
random_search_ann(X, y)

RANDOM SEARCH ANN EXPERIMENT
Fitting 5 folds for each of 100 candidates, totalling 500 fits


2021-11-07 18:14:48.645775: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 18:14:48.649741: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 18:14:48.649821: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9521    0.9865    0.9690       665
         1.0     0.9628    0.8759    0.9173       266

    accuracy                         0.9549       931
   macro avg     0.9575    0.9312    0.9432       931
weighted avg     0.9552    0.9549    0.9542       931

FBeta Score
0.8920367534456357
Model Performance
Accuracy = 0.9549%.
AUC ROC = 0.9847%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9740    0.9740    0.9740       577
         1.0     0.4444    0.4444    0.4444        27

    accuracy                         0.9503       604
   macro avg     0.7092    0.7092    0.7092       604
weighted avg     0.9503    0.9503    0.9503       604

FBeta Score
0.4444444444444444
Model Performance
Accuracy = 0.9503%.
AUC ROC = 0.7760%.
**************


*****************************************************************************************
## Best Random Search Param for ANN
{}

### SKFold + SMOTE (Grid Search)

In [16]:
def grid_search_ann(X, y):
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    
    ann_grid = dict(learn_rate=learn_rate,momentum=momentum,batch_size=batch_size, epochs=epochs)
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    over = SMOTE(sampling_strategy=0.2, random_state = 42)
    under = RandomUnderSampler(sampling_strategy=0.4, random_state = 42)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X_train, y_train = pipeline.fit_resample(X_train, y_train)
    
    ann = KerasClassifier(build_fn=create_model, verbose=0)
    ann_grid = GridSearchCV(estimator = ann, param_grid = ann_grid, n_jobs=-1, cv=skf, scoring='roc_auc')
    ann_grid.fit(X_train, y_train)
    
    ann_best_grid = ann_grid.best_estimator_
    ann_grid_accuracy = evaluate(ann_best_grid, X_train, y_train, X_test, y_test)
    print("Best Params Grid Search for ANN")
    print(ann_grid.best_params_)
    return ann_best_grid

In [17]:
ann_best_grid = grid_search_ann(X, y)

TRAIN
Classification report
              precision    recall  f1-score   support

         0.0     0.9589    0.9835    0.9710       665
         1.0     0.9558    0.8947    0.9243       266

    accuracy                         0.9581       931
   macro avg     0.9574    0.9391    0.9477       931
weighted avg     0.9581    0.9581    0.9577       931

FBeta Score
0.9063214013709062
Model Performance
Accuracy = 0.9581%.
AUC ROC = 0.9905%.
****************************************************************************************************
TEST
Classification report
              precision    recall  f1-score   support

         0.0     0.9735    0.9532    0.9632       577
         1.0     0.3077    0.4444    0.3636        27

    accuracy                         0.9305       604
   macro avg     0.6406    0.6988    0.6634       604
weighted avg     0.9437    0.9305    0.9364       604

FBeta Score
0.40816326530612246
Model Performance
Accuracy = 0.9305%.
AUC ROC = 0.7996%.
*************

[CV] END batch_size=40, epochs=50, learn_rate=0.2, momentum=0.2; total time=   3.8s
[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   3.6s
[CV] END batch_size=100, epochs=100, learn_rate=0.01, momentum=0.9; total time=   4.2s
[CV] END batch_size=10, epochs=100, learn_rate=0.3, momentum=0.4; total time=  15.7s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.9; total time=  15.9s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   1.8s
[CV] END batch_size=20, epochs=10, learn_rate=0.3, momentum=0.6; total time=   2.6s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=  16.3s
[CV] END batch_size=20, epochs=10, learn_rate=0.01, momentum=0.9; total time=   2.5s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.8; total time=   3.3s
[CV] END batch_size=80, epochs=10, learn_rate=0.1, momentum=0.2; total time=   2.3s
[CV] END batch_size=80, epochs=10, learn_rate=0.1, momentum=0.2;

[CV] END batch_size=40, epochs=10, learn_rate=0.1, momentum=0.8; total time=   3.0s
[CV] END batch_size=40, epochs=100, learn_rate=0.3, momentum=0.8; total time=   7.5s
[CV] END batch_size=40, epochs=100, learn_rate=0.3, momentum=0.8; total time=   6.3s
[CV] END batch_size=100, epochs=50, learn_rate=0.01, momentum=0.4; total time=   2.4s
[CV] END batch_size=100, epochs=10, learn_rate=0.01, momentum=0.2; total time=   3.0s
[CV] END batch_size=100, epochs=10, learn_rate=0.01, momentum=0.2; total time=   2.4s
[CV] END batch_size=10, epochs=100, learn_rate=0.2, momentum=0.2; total time=  16.8s
[CV] END batch_size=80, epochs=50, learn_rate=0.01, momentum=0.0; total time=   2.7s
[CV] END batch_size=10, epochs=50, learn_rate=0.001, momentum=0.0; total time=   9.1s
[CV] END batch_size=100, epochs=100, learn_rate=0.001, momentum=0.2; total time=   3.1s
[CV] END batch_size=100, epochs=50, learn_rate=0.1, momentum=0.9; total time=   2.1s
[CV] END batch_size=100, epochs=100, learn_rate=0.2, moment

[CV] END batch_size=40, epochs=50, learn_rate=0.2, momentum=0.2; total time=   3.8s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.2; total time=  15.3s
[CV] END batch_size=100, epochs=10, learn_rate=0.2, momentum=0.2; total time=   2.5s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.9; total time=  15.9s
[CV] END batch_size=60, epochs=100, learn_rate=0.1, momentum=0.0; total time=   4.8s
[CV] END batch_size=80, epochs=10, learn_rate=0.2, momentum=0.9; total time=   2.3s
[CV] END batch_size=20, epochs=10, learn_rate=0.3, momentum=0.6; total time=   2.7s
[CV] END batch_size=80, epochs=50, learn_rate=0.2, momentum=0.4; total time=   2.5s
[CV] END batch_size=10, epochs=100, learn_rate=0.1, momentum=0.6; total time=  16.0s
[CV] END batch_size=100, epochs=100, learn_rate=0.1, momentum=0.8; total time=   3.2s
[CV] END batch_size=10, epochs=50, learn_rate=0.3, momentum=0.2; total time=   8.4s
[CV] END batch_size=60, epochs=10, learn_rate=0.3, momentum=0.8; tota